# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 25 2022 9:35AM,249324,16,SHL-8692280,"SHL Pharma, LLC",Released
1,Oct 25 2022 9:30AM,249323,10,8688495,"Snap Medical Industries, LLC",Released
2,Oct 25 2022 9:30AM,249323,10,8688496,"Snap Medical Industries, LLC",Released
3,Oct 25 2022 9:30AM,249323,10,8688497,"Snap Medical Industries, LLC",Released
4,Oct 25 2022 9:30AM,249323,10,8688498,"Snap Medical Industries, LLC",Released
5,Oct 25 2022 9:30AM,249323,10,8688499,"Snap Medical Industries, LLC",Released
6,Oct 25 2022 9:30AM,249323,10,8688500,"Snap Medical Industries, LLC",Released
7,Oct 25 2022 9:30AM,249323,10,8688501,"Snap Medical Industries, LLC",Released
8,Oct 25 2022 9:30AM,249323,10,8688502,"Snap Medical Industries, LLC",Released
9,Oct 25 2022 9:30AM,249323,10,8689651,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,249314,Released,3
16,249315,Released,1
17,249320,Released,7
18,249323,Released,16
19,249324,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249314,NaN,NaN,3.0
249315,NaN,NaN,1.0
249320,NaN,NaN,7.0
249323,NaN,NaN,16.0
249324,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249232,0.0,0.0,1.0
249236,0.0,0.0,2.0
249279,0.0,1.0,0.0
249282,9.0,1.0,1.0
249293,0.0,0.0,62.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249232,0,0,1
249236,0,0,2
249279,0,1,0
249282,9,1,1
249293,0,0,62


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249232,0,0,1
1,249236,0,0,2
2,249279,0,1,0
3,249282,9,1,1
4,249293,0,0,62


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249232,,,1
1,249236,,,2
2,249279,,1,
3,249282,9,1,1
4,249293,,,62


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 25 2022 9:35AM,249324,16,"SHL Pharma, LLC"
1,Oct 25 2022 9:30AM,249323,10,"Snap Medical Industries, LLC"
17,Oct 25 2022 9:27AM,249320,19,"Graystone, LLC"
24,Oct 25 2022 9:04AM,249315,20,Alumier Labs Inc.
25,Oct 25 2022 9:00AM,249313,10,ISDIN Corporation
71,Oct 25 2022 8:58AM,249314,10,"Methapharm, Inc."
74,Oct 25 2022 8:51AM,249311,10,Bio-PRF
82,Oct 25 2022 8:41AM,249308,10,ISDIN Corporation
102,Oct 24 2022 5:44PM,249301,21,"NBTY Global, Inc."
103,Oct 24 2022 5:38PM,249299,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 25 2022 9:35AM,249324,16,"SHL Pharma, LLC",,,1
1,Oct 25 2022 9:30AM,249323,10,"Snap Medical Industries, LLC",,,16
2,Oct 25 2022 9:27AM,249320,19,"Graystone, LLC",,,7
3,Oct 25 2022 9:04AM,249315,20,Alumier Labs Inc.,,,1
4,Oct 25 2022 9:00AM,249313,10,ISDIN Corporation,,,46
5,Oct 25 2022 8:58AM,249314,10,"Methapharm, Inc.",,,3
6,Oct 25 2022 8:51AM,249311,10,Bio-PRF,,5,3
7,Oct 25 2022 8:41AM,249308,10,ISDIN Corporation,,4,16
8,Oct 24 2022 5:44PM,249301,21,"NBTY Global, Inc.",,,1
9,Oct 24 2022 5:38PM,249299,15,"Brookfield Pharmaceuticals, LLC",,,43


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 25 2022 9:35AM,249324,16,"SHL Pharma, LLC",1,,
1,Oct 25 2022 9:30AM,249323,10,"Snap Medical Industries, LLC",16,,
2,Oct 25 2022 9:27AM,249320,19,"Graystone, LLC",7,,
3,Oct 25 2022 9:04AM,249315,20,Alumier Labs Inc.,1,,
4,Oct 25 2022 9:00AM,249313,10,ISDIN Corporation,46,,
5,Oct 25 2022 8:58AM,249314,10,"Methapharm, Inc.",3,,
6,Oct 25 2022 8:51AM,249311,10,Bio-PRF,3,5,
7,Oct 25 2022 8:41AM,249308,10,ISDIN Corporation,16,4,
8,Oct 24 2022 5:44PM,249301,21,"NBTY Global, Inc.",1,,
9,Oct 24 2022 5:38PM,249299,15,"Brookfield Pharmaceuticals, LLC",43,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 25 2022 9:35AM,249324,16,"SHL Pharma, LLC",1,,
1,Oct 25 2022 9:30AM,249323,10,"Snap Medical Industries, LLC",16,,
2,Oct 25 2022 9:27AM,249320,19,"Graystone, LLC",7,,
3,Oct 25 2022 9:04AM,249315,20,Alumier Labs Inc.,1,,
4,Oct 25 2022 9:00AM,249313,10,ISDIN Corporation,46,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 25 2022 9:35AM,249324,16,"SHL Pharma, LLC",1.0,NaN,NaN
1,Oct 25 2022 9:30AM,249323,10,"Snap Medical Industries, LLC",16.0,NaN,NaN
2,Oct 25 2022 9:27AM,249320,19,"Graystone, LLC",7.0,NaN,NaN
3,Oct 25 2022 9:04AM,249315,20,Alumier Labs Inc.,1.0,NaN,NaN
4,Oct 25 2022 9:00AM,249313,10,ISDIN Corporation,46.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 25 2022 9:35AM,249324,16,"SHL Pharma, LLC",1.0,0.0,0.0
1,Oct 25 2022 9:30AM,249323,10,"Snap Medical Industries, LLC",16.0,0.0,0.0
2,Oct 25 2022 9:27AM,249320,19,"Graystone, LLC",7.0,0.0,0.0
3,Oct 25 2022 9:04AM,249315,20,Alumier Labs Inc.,1.0,0.0,0.0
4,Oct 25 2022 9:00AM,249313,10,ISDIN Corporation,46.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1745087,87.0,10.0,9.0
12,249279,0.0,1.0,0.0
15,747890,119.0,0.0,0.0
16,249324,1.0,0.0,0.0
19,498552,8.0,0.0,0.0
20,249315,1.0,0.0,0.0
21,249301,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1745087,87.0,10.0,9.0
1,12,249279,0.0,1.0,0.0
2,15,747890,119.0,0.0,0.0
3,16,249324,1.0,0.0,0.0
4,19,498552,8.0,0.0,0.0
5,20,249315,1.0,0.0,0.0
6,21,249301,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,87.0,10.0,9.0
1,12,0.0,1.0,0.0
2,15,119.0,0.0,0.0
3,16,1.0,0.0,0.0
4,19,8.0,0.0,0.0
5,20,1.0,0.0,0.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,87.0
1,12,Released,0.0
2,15,Released,119.0
3,16,Released,1.0
4,19,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,9.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,10.0,1.0,0.0,0.0,0.0,0.0,0.0
Released,87.0,0.0,119.0,1.0,8.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Released,87.0,0.0,119.0,1.0,8.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Released,87.0,0.0,119.0,1.0,8.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()